In [10]:
import OpenAI from 'openai';

function getCurrentWeather({ location, unit = 'fahrenheit' }) {
  const weather_info = {
    location: location,
    temperature: '72',
    unit: unit,
    forecast: ['sunny', 'windy'],
  };
  return JSON.stringify(weather_info);
}

const tools = [
  {
    type: 'function',
    function: {
      name: 'getCurrentWeather',
      description: 'Get the current weather in a given location',
      parameters: {
        type: 'object',
        properties: {
          location: {
            type: 'string',
            description: 'The city and state, e.g. San Francisco, CA',
          },
          unit: { type: 'string', enum: ['celsius', 'fahrenheit'] },
        },
        required: ['location'],
      },
    },
  },
];

const messages = [
  {
    role: 'user',
    content: '日本的天气怎么样',
  },
];

const openai = new OpenAI({
  baseURL: 'https://api.shellgpt.top/v1',
});

const result = await openai.chat.completions.create({
  model: 'gpt-3.5-turbo',
  messages,
  tools,
});
// console.log(result.choices[0]);

messages.push(result.choices[0].message);

const functions = {
  getCurrentWeather: getCurrentWeather,
};

const cell = result.choices[0].message.tool_calls[0];
const functionInfo = cell.function;
const functionName = functionInfo.name;
const functionParams = functionInfo.arguments;
const functionResult = functions[functionName](functionParams);

messages.push({
  tool_call_id: cell.id,
  role: 'tool', // 手动执行函数
  name: functionName,
  content: functionResult,  // 执行函数结果
});

console.log('messages :', messages);
// 将所有的对话再次给到 openai
const response = await openai.chat.completions.create({
  model: 'gpt-3.5-turbo',
  messages,
});
console.log(response.choices[0].message);


messages : [
  { role: "user", content: "日本的天气怎么样" },
  {
    role: "assistant",
    content: null,
    tool_calls: [
      {
        id: "call_KfvtuiSBlxfddHPKz6kmYPNX",
        type: "function",
        function: {
          name: "getCurrentWeather",
          arguments: '{"location":"Tokyo, Japan","unit":"celsius"}'
        }
      }
    ],
    refusal: null
  },
  {
    tool_call_id: "call_KfvtuiSBlxfddHPKz6kmYPNX",
    role: "tool",
    name: "getCurrentWeather",
    content: '{"temperature":"72","unit":"fahrenheit","forecast":["sunny","windy"]}'
  }
]
[
  {
    index: 0,
    message: {
      role: "assistant",
      content: "日本的天气情况是晴朗多风，目前温度为摄氏22度（华氏72度）。祝您有一个愉快的一天！",
      refusal: null
    },
    logprobs: null,
    finish_reason: "stop"
  }
]


: 